In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np 
import pandas as pd
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras import layers
import os

In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
from tensorflow.keras.applications import ResNet50

In [4]:
keras.backend.clear_session()

In [5]:
model = ResNet50(weights = 'imagenet', include_top = False, input_shape = (32,32,3))

In [6]:
model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 38, 38, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 16, 16, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 16, 16, 64)   256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [7]:
model.trainable = False

In [8]:
model1 = model.output
model1 = Flatten()(model1)
model1 = Dense(1000, activation='relu', name = 'd1', trainable = True)(model1)
model1 = Dense(500, activation='relu', name = 'd2', trainable = True)(model1)
model1 = Dense(250, activation = 'relu', name ='d3', trainable = True)(model1)
model1 = Dense(125, activation='relu', name = 'd4', trainable = True)(model1)
model1 = Dense(62, activation = 'relu', name ='d5', trainable = True)(model1)
model1 = Dense(10, activation='softmax', name ='preds', trainable = True)(model1)

In [9]:
model1.trainable = True

In [10]:
modelf = Model(inputs = model.input, outputs = model1)

In [11]:
modelf.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 38, 38, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 16, 16, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 16, 16, 64)   256         conv1_conv[0][0]                 
______________________________________________________________________________________________

In [12]:
images_path = r'C:\Users\offic\Documents\GitStuff\NOTEMNIST\notMNIST_large'

In [13]:
classes = os.listdir(images_path)

In [16]:
classes


['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J']

In [17]:
datagen = ImageDataGenerator(images_path,
                             zoom_range = [0.6, 1.9], 
                             width_shift_range=0.3, 
                             height_shift_range=0.3,
                             fill_mode='constant', 
                             cval=190,
                             validation_split = 0.2,
                             )

In [18]:
train_gen = datagen.flow_from_directory(images_path,
        target_size=(32, 32),
        batch_size=32,
        subset = 'training',
        classes = classes
        )

Found 423295 images belonging to 10 classes.


In [19]:
test_gen = datagen.flow_from_directory(images_path,
        target_size=(32, 32),
        batch_size=32,
        subset = 'validation',
        classes = classes
        )

Found 105819 images belonging to 10 classes.


In [20]:
train_steps = train_gen.samples // 32
val_steps = test_gen.samples // 32

In [21]:
modelf.compile(optimizer= 'adam', loss='categorical_crossentropy', metrics = ['accuracy'])

In [22]:
es = EarlyStopping(monitor = 'val_loss',
                   patience = 5,
                   mode = 'auto'
                   )
reduce_lr = ReduceLROnPlateau(monitor = 'val_loss',
                               patience = 2,
                              mode = 'auto'
                              )

checkpoint_filepath = r'C:\Users\offic\Documents\GitStuff\NOTEMNIST\resnet50 check points'
ckpts = tf.keras.callbacks.ModelCheckpoint(
    filepath = checkpoint_filepath,
    save_weights_only = False,
    monitor = 'val_loss',
    mode = 'auto',
    save_best_only = True)


In [25]:
modelf.fit(train_gen,
           epochs = 500,
           validation_data = test_gen,
           steps_per_epoch = train_steps,
           validation_steps = val_steps,
           callbacks = [es, reduce_lr, ckpts] 
           )

Epoch 1/500
13227/13227 [==============================] - 403s 30ms/step - loss: 0.9426 - accuracy: 0.6852 - val_loss: 0.8803 - val_accuracy: 0.7089 - lr: 0.0010
Epoch 2/500
13227/13227 [==============================] - 440s 33ms/step - loss: 0.9375 - accuracy: 0.6879 - val_loss: 0.8754 - val_accuracy: 0.7087 - lr: 0.0010
Epoch 3/500
13227/13227 [==============================] - 342s 26ms/step - loss: 0.9339 - accuracy: 0.6892 - val_loss: 0.8769 - val_accuracy: 0.7069 - lr: 0.0010
Epoch 4/500
13227/13227 [==============================] - 378s 29ms/step - loss: 0.9271 - accuracy: 0.6915 - val_loss: 0.8624 - val_accuracy: 0.7136 - lr: 0.0010
Epoch 5/500
13227/13227 [==============================] - 519s 39ms/step - loss: 0.9243 - accuracy: 0.6923 - val_loss: 0.8588 - val_accuracy: 0.7138 - lr: 0.0010
Epoch 6/500
13227/13227 [==============================] - 380s 29ms/step - loss: 0.9176 - accuracy: 0.6953 - val_loss: 0.8592 - val_accuracy: 0.7143 - lr: 0.0010
Epoch 7/500
13227/1322

13227/13227 [==============================] - 400s 30ms/step - loss: 0.8259 - accuracy: 0.7233 - val_loss: 0.7723 - val_accuracy: 0.7413 - lr: 1.0000e-06
Epoch 30/500
13227/13227 [==============================] - 362s 27ms/step - loss: 0.8261 - accuracy: 0.7231 - val_loss: 0.7765 - val_accuracy: 0.7407 - lr: 1.0000e-06
Epoch 31/500
13227/13227 [==============================] - 382s 29ms/step - loss: 0.8277 - accuracy: 0.7226 - val_loss: 0.7756 - val_accuracy: 0.7401 - lr: 1.0000e-06
Epoch 32/500
13227/13227 [==============================] - 350s 26ms/step - loss: 0.8233 - accuracy: 0.7240 - val_loss: 0.7751 - val_accuracy: 0.7400 - lr: 1.0000e-07
Epoch 33/500
13227/13227 [==============================] - 345s 26ms/step - loss: 0.8261 - accuracy: 0.7234 - val_loss: 0.7731 - val_accuracy: 0.7411 - lr: 1.0000e-07
Epoch 34/500
13227/13227 [==============================] - 345s 26ms/step - loss: 0.8264 - accuracy: 0.7230 - val_loss: 0.7797 - val_accuracy: 0.7424 - lr: 1.0000e-08
